In [1]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-unaybkd_/farm-haystack_2f760719c893438e833a7c145e34d38e
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-unaybkd_/farm-haystack_2f760719c893438e833a7c145e34d38e
  Resolved https://github.com/deepset-ai/haystack.git to commit 081b886aa137cca313562df715be85cee33df5f0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torch-1.10.2-cp38-cp38-manylinux1_x86_64.whl (881.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 13.8 MB/s eta 0:00:00m eta 0:00:010:00:01


  Using cached torchvision-0.12.0-cp38-cp38-manylinux1_x86_64.whl (21.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 16.1 MB/s eta 0:00:00m eta 0:00:010:00:01


  Created wheel for farm-haystack: filename=farm_haystack-1.4.0-py3-none-any.whl size=524692 sha256=707a83cbd189a4c0ebf0f2ea33de2ebad645bbe8d29ab1921a60d4043dbef783
  Stored in directory: /tmp/pip-ephem-wheel-cache-qbpb_ocb/wheels/97/0d/ae/77cde17929fbf66c8320f19b30789acfe52e2312bb1d125be1
Successfully built farm-haystack
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
  Attempting uninstall: farm-haystack
    Found existing installation: farm-haystack 1.3.0
    Uninstalling farm-haystack-1.3.0:
      Successfully uninstalled farm-haystack-1.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [1]:
from haystack.nodes import DensePassageRetriever
from haystack.utils import fetch_archive_from_http
from haystack.document_stores import InMemoryDocumentStore

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [2]:
from haystack.nodes import PDFToTextConverter
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.nodes import FARMReader, ElasticsearchRetriever
from haystack.pipelines import ExtractiveQAPipeline, DocumentSearchPipeline
from haystack.nodes import PreProcessor
from haystack import Document

In [3]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [4]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_big"

In [5]:
retriever = DensePassageRetriever(
    document_store=InMemoryDocumentStore(),
    query_embedding_model=query_model,
    passage_embedding_model=passage_model,
    max_seq_len_query=64,
    max_seq_len_passage=400,
    use_gpu=False
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find IIC/dpr-spanish-question_encoder-allqa-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Automatically detected language from language model name: spanish
INFO - haystack.modeling.model.language_model -  Loaded IIC/dpr-spanish-question_encoder-allqa-base
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find IIC/dpr-spanish-passage_e

In [6]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=test_filename,
    n_epochs=1,
    batch_size=4,
    grad_acc_steps=2,
    save_dir=save_dir,
    evaluate_every=6,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=1,
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /|\  /w\   /w\   /w\   /|\  /w\   /|\  /|\  /|\  /w\   /w\   /w\   /w\   /|\  /|\  /|\
INFO - haystack.modeling.data_handler.data_silo -  /'\   /'\   /'\   / \   /'\   / \ 

INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5723711340206186
INFO - haystack.modeling.evaluation.eval -  average_rank: 0.96
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.8994    0.8994    0.8994       169
     positive     0.3200    0.3200    0.3200        25

     accuracy                         0.8247       194
    macro avg     0.6097    0.6097    0.6097       194
 weighted avg     0.8247    0.8247    0.8247       194

Train epoch 0/0 (Cur. train loss: 12.0913):  92%|▉| 24/26 [02:34<00:10,  5.15s/i
Evaluating: 100%|█████████████████████████████████| 7/7 [00:08<00:00,  1.24s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 24 BATCHES *****
***************************************************
\\|//